# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка.  
Нужно провести обработку и анализ предоставленных данных и ответить на вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Входные данные от банка — статистика о платёжеспособности клиентов, файл "dada.csv": 
- `children` — количество детей в семье 
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Получение данных

In [1]:
# Импортируем библиотеку pandas
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from pymystem3 import Mystem
m = Mystem()

In [2]:
# передаем переменной data значения файла данных типа .csv 
try:
    data = pd.read_csv('data.csv')
except:
    # если не найдена локальная верcия идём в сеть
    data = pd.read_csv('*****')

In [3]:
# общая информация по импортированным данным
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Посмотрим на первые 5 строк наших данных
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Данные представляют собой таблицу с 12 колонками и 21525 строками. В двух колонках, "days_employed" и "otal_income", имеются пропуски, количество которых совпадает (возможна связь?). Колонка "days_employed" содержит отрицательные значения (?!?!). Строго количественные данные в колонках "days_employed" и "total_income". Для ответа на поставленные вопросы, "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", необходимо проанализировать данные столбцов: 'children', 'family_status', 'debt'. Однако, в рамках работы, рассмотрим влияние всех представленных факторов на финансовую дисциплину клиентов. Прежде чем дать отвыты на вопросы исследования, необходимо изучить данные, выявить ошибки и постараться их исправить, а также привести данные к более удобному формату для последующей группировки и анализа. Под ошибками будем понимать: пропуски, анамальные значания, явные и неявные дубликаты.  

## Предобработка данных

### Обработка пропусков

Как было указано ранее, возмодна связь между пропусками в колонках "days_employed" и "total_income".
Проверим это. Найдем кол-во пропусков в каждом из столбцов, а также количество сткок, где пропущены оба значения.

In [5]:
print ("Количество пропусков 'days_employed' = ", data['days_employed'].isna().sum())
print ("Количество пропусков 'total_income' = ", data['total_income'].isna().sum())
print ("Количество строк, где промущены оба значения", data.loc[data['days_employed'].isna()]['total_income'].isna().sum())

Количество пропусков 'days_employed' =  2174
Количество пропусков 'total_income' =  2174
Количество строк, где промущены оба значения 2174


Из полученных значений можно сделать вывод, что существует связь между пропусками в колонках "days_employed" и "total_income".

In [6]:
# посмотрим на первые 10 строк с пропусками и оценим наличие каких-либо видимых закономерностей
data.loc[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Видимых закономерностей не установлено. Так как в рамках поставленной задачи, значения 'days_employed' не окажут влияния на результат исследования, не будем проводить дальнейшее исследование по поиску взаимосвязей.

**Роботаем с пропусками в столбце 'days_employed'.**
Очевидно, что трудовой стаж связан с возрастом клиента.
Можно сгруппировать клиентов по возрастным группам и использовать медианное значение стажа для пропусков внутри группы.
Прежде всего, необходимо решить, что делать с отрицательными значениями стажа и с оценить кол-во ошибочных данных.
На данном этапе, под ошибками имееется ввиду стаж, например, превышающий разумные пределы. Так для записи под номером 4  заявлен стаж 340266.072047 дней. Что, при средней продолжительности года в 365/366 дней, составит около 930 лет.
На сегодняшний день, пенсионный возраст в России: 60 лет — для женщин и 65 лет — для мужчин.
Будем полагать, что речь идёт о подтвержденном стаже, т.е. в соотвествии с ТК РФ начало тредовой деятельности с 16 лет.
Таким образом, наиболее вероятный максимальны стаж 60 - 16 = 44 года, или порядка 16 000 дней.

In [7]:
# рассмотрим положительны стаж детельно
print ('Минимальное значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].min()))
print ('Максимальное значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].max()))
print ('Количество значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].count()))

Минимальное значение положительного стажа:  328728
Максимальное значение положительного стажа:  401755
Количество значение положительного стажа:  3445


Вывод: Все данные с положительным стажем ошибочны. Максимальное значение стажа данных - 401744, что в 25 раз больше предполагаемого максимально стажа 16000. Возможно, эти данные указаны в часах, т.е. с мультипликатором 24.

In [8]:
# посмотрим первые 10 записей со стажем > 0
data[data['days_employed'] > 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


Среди первых 10, положительный стаж только у клиентов категории "пенсионер". Проверим все категории на наличие/отсутствие такого стажа.

In [9]:
data[data['days_employed'] > 0].groupby('income_type')['days_employed'].count()

income_type
безработный       2
пенсионер      3443
Name: days_employed, dtype: int64

Проверим группы на наличие стажа = 0

In [10]:
data[data['days_employed'] == 0].groupby('income_type')['days_employed'].count()

Series([], Name: days_employed, dtype: int64)

Проверим группы на наличие стажа < 0

In [11]:
data[data['days_employed'] < 0].groupby('income_type')['days_employed'].count()

income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

In [12]:
# Заменим значения 'days_employed', которые больше 0 на 'days_employed'/ 24
data.loc[(data['days_employed'] > 0), 'days_employed'] = data['days_employed']/24 
# проверка
print ('Минимальное значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].min()))
print ('Максимальное значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].max()))
print ('Количество значение положительного стажа: ', int(data.loc[data['days_employed'] > 0]['days_employed'].count()))

Минимальное значение положительного стажа:  13697
Максимальное значение положительного стажа:  16739
Количество значение положительного стажа:  3445


In [13]:
# рассмотрим отрицательный стаж 
print ('Минимальное значение отрицательного стажа: ', int(data.loc[data['days_employed'] < 0]['days_employed'].min()))
print ('Максимальное значение отрицательного стажа: ', int(data.loc[data['days_employed'] < 0]['days_employed'].max()))
print ('Количество значение отрицательного стажа: ', int(data.loc[data['days_employed'] < 0]['days_employed'].count()))

Минимальное значение отрицательного стажа:  -18388
Максимальное значение отрицательного стажа:  -24
Количество значение отрицательного стажа:  15906


Опираясь на эти значения и максимальный теоретический стаж, допускаем, что ошибка только в знаке данных.

In [14]:
# Заменим значения 'days_employed', которые меньше 0 на модуль'days_employed'
data.loc[(data['days_employed'] < 0), 'days_employed'] = data['days_employed'].abs()
# проверка
print ('Минимальное значение отрицательного стажа: ', data.loc[data['days_employed'] < 0]['days_employed'].min())
print ('Максимальное значение отрицательного стажа: ', data.loc[data['days_employed'] < 0]['days_employed'].max())
print ('Количество значение отрицательного стажа: ', data.loc[data['days_employed'] < 0]['days_employed'].count())

Минимальное значение отрицательного стажа:  nan
Максимальное значение отрицательного стажа:  nan
Количество значение отрицательного стажа:  0


Для выделения возрастных категорий необходимо убедиться в отсутствии онибок в колонке 'dob_years'

In [15]:
print ('Минимальный возраст клиента', data['dob_years'].min())
print ('Максимальный возраст клиента', data['dob_years'].max())

Минимальный возраст клиента 0
Максимальный возраст клиента 75


Видим, что имеет место ошибка в возрасте. Оценим количество строк с возрастом 0.

In [16]:
data[data['dob_years'] == 0]['dob_years'].count()

101

In [17]:
# посмотрим на первые 10 строк, для которых 'dob_years' = 0
data[(data['dob_years'] == 0)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,15453.312833,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,15422.689043,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [18]:
# оценим кол-во записей, в которых присутствуют пропуски и возраст заёмщика равен 0
data.loc[(data['dob_years'] == 0) & data['days_employed'].isna()]['dob_years'].count()

10

In [19]:
# всего 10 строк. посмотим их
data[(data['dob_years'] == 0) & data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
2284,0,NaN,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,недвижимость
4064,1,NaN,0,среднее,1,гражданский брак,1,M,компаньон,0,NaN,ремонт жилью
5014,0,NaN,0,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка недвижимости
6411,0,NaN,0,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свадьба
6670,0,NaN,0,Высшее,0,в разводе,3,F,пенсионер,0,NaN,покупка жилой недвижимости
8574,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,недвижимость
12403,3,NaN,0,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью
13741,0,NaN,0,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,на проведение свадьбы
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье


10 записей из 21525 составляют менее 0.05% при заполнения медианными знечениями ощутимого влияния на конечный результт не окажут. Можно удалить

In [20]:
# индексы строк для удаления
row_indx = data.index[(data['dob_years'] == 0) & data['days_employed'].isna()]
# удаляем 
data = data.drop(row_indx)
# проверяем
data.loc[(data['dob_years'] == 0) & data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [21]:
# группируем строки с возрастом равным 0 по 'income_type'
data[data['dob_years'] == 0].groupby('income_type')['income_type'].count()

income_type
госслужащий     6
компаньон      18
пенсионер      17
сотрудник      50
Name: income_type, dtype: int64

In [22]:
# посмотрим на доли
with pd.option_context('display.float_format','{:.2%}'.format):
    display(data.groupby('income_type').apply(lambda group : (group['dob_years'] == 0).mean()))

income_type
безработный       0.00%
в декрете         0.00%
госслужащий       0.41%
компаньон         0.35%
пенсионер         0.44%
предприниматель   0.00%
сотрудник         0.45%
студент           0.00%
dtype: float64

Доли несущественны и не окажут значимого влияния на результат исследования.

In [23]:
# меняем все нулевые dob на медианные значения в группах 'income_type'
for i in data['income_type'].unique():
    dob_mediana = data.loc[(data['income_type'] == i) & data['dob_years'] != 0]['dob_years'].median()
    data.loc[(data['income_type'] == i) & (data['dob_years'] == 0), 'dob_years'] = dob_mediana 
# проверка
print ('Минимальный возраст клиента', data['dob_years'].min())
print ('Максимальный возраст клиента', data['dob_years'].max())

Минимальный возраст клиента 19.0
Максимальный возраст клиента 75.0


Посмотрим на наличее ошибок во взаимосвязи стажа и возраста клиентов.

In [24]:
# Записи для которых стаж больше возраста.
print ('Число записей по категориям для которых стаж больше возраста:', end = ' ')
print (data.loc[data['dob_years'] < data['days_employed']/366]['income_type'].count())
print (data.loc[data['dob_years'] < data['days_employed']/366]['income_type'].value_counts())

Число записей по категориям для которых стаж больше возраста: 53
пенсионер      51
безработный     2
Name: income_type, dtype: int64


In [25]:
# Посмотрим на первые 5 записей для которых стаж больше возраста
data.loc[data['dob_years'] < data['days_employed']/366].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
157,0,14517.251167,38.0,среднее,1,женат / замужем,0,F,пенсионер,1,113560.650035,сделка с автомобилем
751,0,16281.477669,41.0,среднее,1,женат / замужем,0,M,пенсионер,0,151898.693438,операции со своей недвижимостью
776,0,15222.356680,38.0,среднее,1,женат / замужем,0,F,пенсионер,0,73859.425084,покупка недвижимости
1242,0,13948.510826,22.0,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования
1383,0,14741.783820,37.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости


Из данных видно, что данные по стажу близки к теоретическому максимальному стажу для группы 'пенсионер', а возраст не соответствует "days_employed" и "income_type". Т.е. будем полагать, что этой группы записей ошибочным являются данные столбца 'dob_years' заменим их на расчётный возраст по формуле- (стаж/365 + 16)

In [26]:
data.loc[data['dob_years'] < data['days_employed']/366, 'dob_years'] = data['days_employed'] / 365 + 16
# проверка
print ('Число записей по категориям для которых стаж больше возраста:', end = ' ')
print (data.loc[data['dob_years'] < data['days_employed']/366]['income_type'].count())
if data.loc[data['dob_years'] < data['days_employed']/366]['income_type'].count() != 0:
    print (data.loc[data['dob_years'] < data['days_employed']/366]['income_type'].value_counts())

Число записей по категориям для которых стаж больше возраста: 0


In [27]:
# Записи для которых стаж больше трудоспособного возраста.
# Здесь трудоспособным возрастом будем считать 14 лет.
print ('Число записей по категориям для которых стаж больше трудоспособного возраста:', end = ' ')
print (data.loc[data['dob_years'] -14 < data['days_employed']/366]['income_type'].count())
print (data.loc[data['dob_years'] -14 < data['days_employed']/366]['income_type'].value_counts())

Число записей по категориям для которых стаж больше трудоспособного возраста: 828
пенсионер      811
сотрудник       11
компаньон        4
госслужащий      2
Name: income_type, dtype: int64


In [28]:
# Посмотрим на первые 5 записей для которых стаж больше трудоспособного возраста
data.loc[data['dob_years'] - 14 < data['days_employed']/366].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18,0,16678.380705,53.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
98,0,15204.425239,54.0,высшее,0,женат / замужем,0,F,пенсионер,0,199707.298524,покупка жилья для сдачи
178,0,16488.681725,59.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,87268.778786,приобретение автомобиля
206,0,15163.536736,54.0,Среднее,1,женат / замужем,0,F,пенсионер,0,267172.709200,получение образования
217,0,16023.746735,54.0,среднее,1,гражданский брак,1,F,пенсионер,0,269589.005129,строительство собственной недвижимости


Видно, что для этих записей ошибка несущественна. Т.е. стаж примерно соотвествуеет возрасту.

In [29]:
# посмотрим распределение возрастов в этой группе
data.loc[data['dob_years'] - 14 < data['days_employed']/366]['dob_years'].value_counts()

55.0    99
54.0    93
52.0    88
56.0    87
53.0    82
51.0    73
57.0    65
50.0    62
58.0    34
49.0    30
48.0    21
59.0    18
47.0    15
46.0    14
44.0    10
45.0     9
42.0     7
43.0     7
41.0     4
36.0     2
61.0     2
39.0     2
40.0     2
29.0     1
38.0     1
Name: dob_years, dtype: int64

In [30]:
# Оценим максимальную ошибку по группе
data1 = data.loc[data['dob_years'] - 14 < data['days_employed']/366]
print ('Минимальный возраст начала трудового стажа', (data1['dob_years']-data1['days_employed']/366).max())


Минимальный возраст начала трудового стажа 13.999647344011173


Так как максимальная ошибка близка к теоретически принятому значению в 14 лет, оставляем данные без изменений.

Установим возрастные группы для определения стажа в пропусках.

In [31]:
# "ширина" группы 10лет, т.е., например, группа 3 это клиенты с dob от 30 до 39 лет
data['age_groups'] = (data ['dob_years'] + 1) // 10
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groups
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,4.0
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,3.0
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,3.0
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,3.0
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,5.0


In [32]:
# посмотрим распределение по возрастным группам
data['age_groups'] = data['age_groups'].astype('int')
data.groupby('age_groups')['age_groups'].count()


age_groups
2    2631
3    5620
4    5490
5    4770
6    2748
7     256
Name: age_groups, dtype: int64

In [33]:
# посмотрим сколько пропусков в меньшей, 7-ой группе
data_miss_in7 = data.loc[(data['age_groups'] == 7) & (data['days_employed'].isna() == True)]['age_groups'].count()
print('В 7-ой группе', data_miss_in7,'пропусков, что составляет', data_miss_in7/256*100, '% от общего число в группе' )

В 7-ой группе 16 пропусков, что составляет 6.25 % от общего число в группе


Доля незначительна, оставляем группы без изменений.

Обработаем пропуски в колонке 'days_employed'. Меняем на медианные значения внутри групп.

In [34]:
for i in data['age_groups'].unique():
    gr_median = data.loc[(data['age_groups'] == i) & data['days_employed'].isna() == False]['days_employed'].median()
    data.loc[(data['age_groups'] == i) & (data['days_employed'].isna()), 'days_employed'] = gr_median

Проверяем пропуски по колонке "days_employed"

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21515 non-null  int64  
 1   days_employed     21515 non-null  float64
 2   dob_years         21515 non-null  float64
 3   education         21515 non-null  object 
 4   education_id      21515 non-null  int64  
 5   family_status     21515 non-null  object 
 6   family_status_id  21515 non-null  int64  
 7   gender            21515 non-null  object 
 8   income_type       21515 non-null  object 
 9   debt              21515 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21515 non-null  object 
 12  age_groups        21515 non-null  int32  
dtypes: float64(3), int32(1), int64(4), object(5)
memory usage: 2.7+ MB


In [36]:
# вспомогательный столбец "age_groups" нам больше не нужен, удалим его
data.drop('age_groups', axis=1, inplace=True)

**Роботаем с пропусками в столбце 'total_income'.**
Бедем менять на медиану для каждой из категорий 'income_type'. Выбор медианного значения обусловлено меньшему влиянию "выбросов", по сравлению с арифметическим средним.
Создадим цикл по значениям 'income_type'.

In [37]:
# посмотрим на уникальные значения 'income_type'
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [38]:
# создадим цикл для перебора категорий 'income_type'
for i in data['income_type'].unique():
    # находим медиану для категории i
    ti_median = data.loc[data['income_type'] == i]['total_income'].median()
    # меняем пропуски на медиану, при совпадении категории
    data.loc[(data['income_type'] == i) & data['total_income'].isna(), 'total_income'] = ti_median  
# проверка
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21515 non-null  int64  
 1   days_employed     21515 non-null  float64
 2   dob_years         21515 non-null  float64
 3   education         21515 non-null  object 
 4   education_id      21515 non-null  int64  
 5   family_status     21515 non-null  object 
 6   family_status_id  21515 non-null  int64  
 7   gender            21515 non-null  object 
 8   income_type       21515 non-null  object 
 9   debt              21515 non-null  int64  
 10  total_income      21515 non-null  float64
 11  purpose           21515 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.6+ MB


Проверим ошибки в полях 'children',  'education_id',  'family_status_id',  'gender',  'debt'

In [39]:
# Крайние значения по столбцу 'children'
print ('Минимальное количество детей', data['children'].min())
print ('Максимальное количество детей', data['children'].max())

Минимальное количество детей -1
Максимальное количество детей 20


In [40]:
# количество записей со значением в колонке 'children' -1
print ("Количество записей со значением -1 в колонке 'children' :", data.loc[data['children'] == -1]['children'].count())

Количество записей со значением -1 в колонке 'children' : 47


In [41]:
# Вероятно, ошибка свызана с указанием тире в поле дети. Меняем на 1.
data.loc[(data['children'] == -1), 'children'] = data['children'].abs()
# проверка
print ("Количество записей со значением -1 в колонке 'children' :", data.loc[data['children'] == -1]['children'].count())

Количество записей со значением -1 в колонке 'children' : 0


In [42]:
# количество записей со значением в колонке 'children' 20
print ("Количество записей со значением 20 в колонке 'children' :", data.loc[data['children'] == 20]['children'].count())

Количество записей со значением 20 в колонке 'children' : 76


In [43]:
# Вероятно, ошибка, приписан лишний 0. Примем дла этих записей число детей равным 2.
data.loc[(data['children'] == 20), 'children'] = 2
# проверка
print ("Количество записей со значением 20 в колонке 'children' :", data.loc[data['children'] == 20]['children'].count())

Количество записей со значением 20 в колонке 'children' : 0


In [44]:
# Уникальные значения столбца 'children'
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [45]:
# Уникальные значения столбца 'education_id'
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [46]:
# Уникальные значения столбца 'family_status_id'
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [47]:
# Уникальные значения столбца 'gender'
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [48]:
# Проверим кол-во значений 'XNA'
data[data['gender'] == 'XNA']['gender'].count()

1

Всего одна запись. Оставляем без изменения.

In [49]:
# Уникальные значения столбца 'debt'
data['debt'].unique()

array([0, 1], dtype=int64)

**Вывод**

Исходные данные содержали связанные пропуски в столбцах "days_employed" и "total_income".
Пропуски были заменены медианными значениями внутри соответствующих категорий.
Помимо пропусков данные проверены на наличие ананальных значений. Найденные значения исправлены.

### Замена типа данных

Три столбца данных содержат данные типа float. Дробные части данных в этих столбцах не окажут значимого влияния на цели исследования. Преобразуем их в целочисленные значения.

In [50]:
data['days_employed'] = data['days_employed'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [51]:
# проверка
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21515 non-null  int64 
 1   days_employed     21515 non-null  int32 
 2   dob_years         21515 non-null  int32 
 3   education         21515 non-null  object
 4   education_id      21515 non-null  int64 
 5   family_status     21515 non-null  object
 6   family_status_id  21515 non-null  int64 
 7   gender            21515 non-null  object
 8   income_type       21515 non-null  object
 9   debt              21515 non-null  int64 
 10  total_income      21515 non-null  int32 
 11  purpose           21515 non-null  object
dtypes: int32(3), int64(4), object(5)
memory usage: 2.4+ MB


In [52]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Произведенные замены типов данных сделали данные столбцов более удобными для визуального восприятия и анализа. Так же уменьшился объем занимаемой памяти.

### Обработка дубликатов

Посмотрим на явные дубликаты

In [53]:
data.duplicated().sum()

54

Удалим дубликаты и обновим индексы

In [54]:
data = data.drop_duplicates().reset_index(drop=True)
# проверка
data.duplicated().sum()

0

Проверим на наличие неявных дубликатов. Их наличие возможно в столбцах со строковыми переменными.
Для этого посмотрим уникальнае значения.

In [55]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [56]:
# приведём свсе к нижнему регистру
data['education'] = data['education'].str.lower()
# проверка
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [57]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [58]:
# хотя повторов и нет, но приведем к нижнему регистру
data['family_status'] = data['family_status'].str.lower()
# проверка
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [59]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [60]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [61]:
# большое разнообразие записей. чтобы исключить оштибку визуального поиска приведем к нижнему регистру.
data['purpose'] = data['purpose'].str.lower()
# проверка
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Удалено 54 явных дубликата. Произведён поиск и исправление неявных дубликатов.

### Лемматизация

В колонке 'purpose' есть скрытые дубликаты. Т.е. одни цели описаны разными словами, например: 'покупка жилья', 'операции с жильем', 'покупка жилья для семьи' и т.д. и т.п. Эти повторы нужно привести к единому виду.

In [62]:
print ('Текущее количество уникальных целей:', len(data['purpose'].unique()))

Текущее количество уникальных целей: 38


Определим частоту использования основ слов в множестве значений 'purpose'

In [63]:
purposes = data['purpose'].unique()
lemmas =' '.join(m.lemmatize(' '.join(purposes)))
from collections import Counter
Counter(lemmas.split())

Counter({'покупка': 10,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

Выбираем наиболее часто встречающиеся значения и запишем их в Series. Таких значний будет 5.

In [64]:
new_name = pd.Series(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'])

In [65]:
# создадим DF с оригинальными наименованиями
names = pd.DataFrame(data = purposes, columns =  ['original_name'])

In [66]:
# функция для нахождения основы слова
def lem_fun(o_name):
    lm = ''.join(m.lemmatize(o_name))
    return lm       

In [67]:
# добавим в 'names' слолбец 'lemma' c основами слов столбца 'original_name'
names['lemma'] = names['original_name'].progress_apply(lem_fun)

100%|██████████████████████████████████████████| 38/38 [01:20<00:00,  2.12s/it]


In [68]:
# проверим, есть ли в леммах оригинальных наименований наши выбранные леммы :
# ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость']
# при нахождении соответствия, записываем его в столбец 'new_name'
for i in new_name:
    for n in range(len(purposes)):#len(purposes)):
        if i in names.loc[n,'lemma']:
            names.loc[n,'new_name'] = i
names

,original_name,lemma,new_name
0,покупка жилья,покупка жилье\n,жилье
1,приобретение автомобиля,приобретение автомобиль\n,автомобиль
2,дополнительное образование,дополнительный образование\n,образование
3,сыграть свадьбу,сыграть свадьба\n,свадьба
4,операции с жильем,операция с жилье\n,жилье
5,образование,образование\n,образование
6,на проведение свадьбы,на проведение свадьба\n,свадьба
7,покупка жилья для семьи,покупка жилье для семья\n,жилье
8,покупка недвижимости,покупка недвижимость\n,недвижимость
9,покупка коммерческой недвижимости,покупка коммерческий недвижимость\n,недвижимость


Видим, что все строки заполнены, а значит наше предположение о 5 основных леммах верно.
Проверим группы по каждой из 5 лемм.

In [69]:
names.loc[names['new_name'] == 'жилье']

,original_name,lemma,new_name
0,покупка жилья,покупка жилье\n,жилье
4,операции с жильем,операция с жилье\n,жилье
7,покупка жилья для семьи,покупка жилье для семья\n,жилье
18,жилье,жилье\n,жилье
27,покупка своего жилья,покупка свой жилье\n,жилье
34,покупка жилья для сдачи,покупка жилье для сдача\n,жилье
36,ремонт жилью,ремонт жилье\n,жилье


Можно выделить новую кетегорию - 'ремонт'

In [70]:
# поменяем значение столбца 'new_name' в строке с индексом 36
names.loc[36, 'new_name'] = 'ремонт'
names.loc[names['new_name'] == 'ремонт']

,original_name,lemma,new_name
36,ремонт жилью,ремонт жилье\n,ремонт


In [71]:
names.loc[names['new_name'] == 'автомобиль']

,original_name,lemma,new_name
1,приобретение автомобиля,приобретение автомобиль\n,автомобиль
14,на покупку подержанного автомобиля,на покупка подержать автомобиль\n,автомобиль
15,на покупку своего автомобиля,на покупка свой автомобиль\n,автомобиль
20,автомобили,автомобиль\n,автомобиль
22,сделка с подержанным автомобилем,сделка с подержанный автомобиль\n,автомобиль
24,автомобиль,автомобиль\n,автомобиль
30,свой автомобиль,свой автомобиль\n,автомобиль
31,сделка с автомобилем,сделка с автомобиль\n,автомобиль
35,на покупку автомобиля,на покупка автомобиль\n,автомобиль


По группе замечаний нет.

In [72]:
names.loc[names['new_name'] == 'образование']

,original_name,lemma,new_name
2,дополнительное образование,дополнительный образование\n,образование
5,образование,образование\n,образование
21,заняться образованием,заниматься образование\n,образование
23,получение образования,получение образование\n,образование
26,получение дополнительного образования,получение дополнительный образование\n,образование
29,получение высшего образования,получение высокий образование\n,образование
32,профильное образование,профильный образование\n,образование
33,высшее образование,высокий образование\n,образование
37,заняться высшим образованием,заниматься высокий образование\n,образование


По группе замечаний нет.

In [73]:
names.loc[names['new_name'] == 'свадьба']

,original_name,lemma,new_name
3,сыграть свадьбу,сыграть свадьба\n,свадьба
6,на проведение свадьбы,на проведение свадьба\n,свадьба
25,свадьба,свадьба\n,свадьба


По группе замечаний нет.

In [74]:
names.loc[names['new_name'] == 'недвижимость']

,original_name,lemma,new_name
8,покупка недвижимости,покупка недвижимость\n,недвижимость
9,покупка коммерческой недвижимости,покупка коммерческий недвижимость\n,недвижимость
10,покупка жилой недвижимости,покупка жилой недвижимость\n,недвижимость
11,строительство собственной недвижимости,строительство собственный недвижимость\n,недвижимость
12,недвижимость,недвижимость\n,недвижимость
13,строительство недвижимости,строительство недвижимость\n,недвижимость
16,операции с коммерческой недвижимостью,операция с коммерческий недвижимость\n,недвижимость
17,строительство жилой недвижимости,строительство жилой недвижимость\n,недвижимость
19,операции со своей недвижимостью,операция со свой недвижимость\n,недвижимость
28,операции с недвижимостью,операция с недвижимость\n,недвижимость


Группы 'недвижимость' и 'жилье' можно объединить и выделить отдельно операции с коммерческой недвижимостью.

In [75]:
# меняем 'жилье' на 'недвижимость', а если в лемме есть 'коммерческий', то меняем на 'недвижимость ком.'
names['new_name'] = names['new_name'].replace('жилье', 'недвижимость')
for n in range(len(purposes)):
    if 'коммерческий' in names.loc[n,'lemma']:
            names.loc[n,'new_name'] = 'недвижимость ком.'
names

,original_name,lemma,new_name
0,покупка жилья,покупка жилье\n,недвижимость
1,приобретение автомобиля,приобретение автомобиль\n,автомобиль
2,дополнительное образование,дополнительный образование\n,образование
3,сыграть свадьбу,сыграть свадьба\n,свадьба
4,операции с жильем,операция с жилье\n,недвижимость
5,образование,образование\n,образование
6,на проведение свадьбы,на проведение свадьба\n,свадьба
7,покупка жилья для семьи,покупка жилье для семья\n,недвижимость
8,покупка недвижимости,покупка недвижимость\n,недвижимость
9,покупка коммерческой недвижимости,покупка коммерческий недвижимость\n,недвижимость ком.


Таблица соответствий готова. Применим ее для замены оригинальных значений 'purpose' на новые.

In [76]:
for i in range(len(purposes)):
    data['purpose'] = data['purpose'].replace(names.loc[i, 'original_name'], names.loc[i, 'new_name'])
# проверка
data['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба',
       'недвижимость ком.', 'ремонт'], dtype=object)

**Вывод**

В исходных данных цели, обозначенные разними словами, зачастую имели одно значение. Лемматизация помогла выделить из 38 разныз целей в исходных данных 6 групп.

### Категоризация данных

**Определим зависимость между количеством детей и случаями задолженности.**

In [77]:
# создадим функцию для подсчёта общего числа записей в столбце 'm' удоблетворяющих критерию 'n'
def total_lines (n, m):
    tl = data.loc[data[m] == n][m].count()
    return tl

In [78]:
# Функция для нахождения случаев задолженности по категории (cret)
def dep(cret):
    data2 = data.loc[:, [cret, 'debt']].groupby(cret).sum()
    data2 ['total_cases'] = data.groupby(cret)[cret].count()
    data2 ['ratio'] =(data2 ['debt'] / data2 ['total_cases'])
    data2 ['div_from_total'] = data2 ['ratio'] - data['debt'].sum() / data['debt'].count()
    data2 = data2.style.format({'ratio':'{:.2%}', 'div_from_total':'{:.2%}'})
    print ('Total debt ratio {:.2%}'.format(data['debt'].sum() / data['debt'].count()))
    return data2

In [79]:
data_children = dep('children')
data_children

Total debt ratio 8.11%


,debt,total_cases,ratio,div_from_total
children,,,,
0,1063,14099,7.54%,-0.57%
1,445,4855,9.17%,1.05%
2,202,2128,9.49%,1.38%
3,27,329,8.21%,0.09%
4,4,41,9.76%,1.64%
5,0,9,0.00%,-8.11%


**Определим зависимость между семейным положением и случаями задолженности.**

In [80]:
data_family_st = dep('family_status')
data_family_st

Total debt ratio 8.11%


,debt,total_cases,ratio,div_from_total
family_status,,,,
в разводе,85,1194,7.12%,-0.99%
вдовец / вдова,63,958,6.58%,-1.54%
гражданский брак,388,4160,9.33%,1.21%
женат / замужем,931,12340,7.54%,-0.57%
не женат / не замужем,274,2809,9.75%,1.64%


**Определим зависимость между уровнем дохода и случаями задолженности.**

Для определения замвисимости случаев задолженности от уроня дохода, необходимо сформировать интервалы по уровню дохода с приемлемой репрезентативностью.

In [81]:
# посмотрим показатели дохода
print ('Минимальный доход ', data['total_income'].min())
print ('Медианный доход   ', int(data['total_income'].median()))
print ('Средний доход     ', int(data['total_income'].mean()))
print ('Максимальны доход ', data['total_income'].max())

Минимальный доход  20667
Медианный доход    142594
Средний доход      165306
Максимальны доход  2265604


Видим широкий разброс дохода. Максимальный больше минимального почти в 110 раз. Значительное превышение среднего над медианой говорит о том, что есть "выбросы" у верхней границы доходов. Будем полагать, что на такой выборке нам будет достаточно разбить клиетнов на 5 групп по уровню дохода. Отсортируем данные по увеличению дохода и определим верхнии границы интервалов.

In [82]:
# сортировка по уровню дохода c сохранением старых индексов
data = data.sort_values(by = 'total_income').reset_index()

In [83]:
# определим интервалы в каждом из которых будет 1/5 от общего колечества данных
# вернём значение дохода для последней записи интервала
for i in range(4):
    n = data['total_income'].count() // 5 + data['total_income'].count() //5 * i
    print ('Интервал', i+1, 'верхняя граница',data.loc[n,'total_income'])

Интервал 1 верхняя граница 98552
Интервал 2 верхняя граница 132121
Интервал 3 верхняя граница 161306
Интервал 4 верхняя граница 214604


In [84]:
# округлим полученные данные и создадим словарь названий интервалов
incom_levels = {
        1 : 'до 100000',
        2 : 'от 100001 до 130000',
        3 : 'от 130001 до 160000',
        4 : 'от 160001 до 220000',
        5 : 'свыше 220001'
}

In [85]:
# функция для нахождение номера интервада по доходу
def incom_id(incom):
    if incom <= 100000:
        return 1
    elif incom > 100000 and incom <= 130000:
        return 2
    elif incom > 130000 and incom <= 160000:
        return 3
    elif incom > 160000 and incom <= 220000:
        return 4
    else:
        return 5

In [86]:
# добавим в табдицу столбец 'incom_id', 
# значения которого получаться в результате применения функции incom_id к столбцу 'total_income'
data ['incom_id'] = data['total_income'].apply(incom_id)

In [87]:
# тут немного доработаем для добавления колонки с названием интервала
data_incom = dep('incom_id').data
data_incom.insert(0, 'incom_level', [incom_levels.get(x) for x in data_incom.index])
data_incom = data_incom.style.format({'ratio':'{:.2%}', 'div_from_total':'{:.2%}'})
data_incom

Total debt ratio 8.11%


,incom_level,debt,total_cases,ratio,div_from_total
incom_id,,,,,
1,до 100000,354,4463,7.93%,-0.18%
2,от 100001 до 130000,328,3871,8.47%,0.36%
3,от 130001 до 160000,391,4407,8.87%,0.76%
4,от 160001 до 220000,384,4712,8.15%,0.04%
5,свыше 220001,284,4008,7.09%,-1.03%


**Определим зависимость между целью кредитаи случаями задолженности.**

In [88]:
# зависимость между целью кредита и случаями задолженности
for i in data['purpose'].unique():
    debt_cases = data.loc[data['purpose'] == i]['debt'].sum()
    debt_ratio = round(debt_cases / total_lines(i, 'purpose') *100, 2)
    print(f"Цель кредита - {i:<18} | случаев задолженности {debt_cases: < 5} | доля по группе {debt_ratio: < 6} % | всего в группе {total_lines(i, 'purpose')}" )

Цель кредита - недвижимость       | случаев задолженности  648  | доля по группе  7.29  % | всего в группе 8889
Цель кредита - образование        | случаев задолженности  370  | доля по группе  9.22  % | всего в группе 4014
Цель кредита - автомобиль         | случаев задолженности  403  | доля по группе  9.35  % | всего в группе 4308
Цель кредита - свадьба            | случаев задолженности  186  | доля по группе  7.97  % | всего в группе 2333
Цель кредита - недвижимость ком.  | случаев задолженности  99   | доля по группе  7.55  % | всего в группе 1311
Цель кредита - ремонт             | случаев задолженности  35   | доля по группе  5.78  % | всего в группе 606


In [89]:
data_purpose = dep('purpose')
data_purpose

Total debt ratio 8.11%


,debt,total_cases,ratio,div_from_total
purpose,,,,
автомобиль,403,4308,9.35%,1.24%
недвижимость,648,8889,7.29%,-0.82%
недвижимость ком.,99,1311,7.55%,-0.56%
образование,370,4014,9.22%,1.11%
ремонт,35,606,5.78%,-2.34%
свадьба,186,2333,7.97%,-0.14%


**Дополнительно.** Хотя это и не требуется в рамках задания, но оценим, на смольно другие представленные в данных факторы влияют на финансовую дисциплину клиента.

In [90]:
data_education = dep('education')
data_education

Total debt ratio 8.11%


,debt,total_cases,ratio,div_from_total
education,,,,
высшее,278,5248,5.30%,-2.82%
начальное,31,282,10.99%,2.88%
неоконченное высшее,68,744,9.14%,1.03%
среднее,1364,15181,8.98%,0.87%
ученая степень,0,6,0.00%,-8.11%


In [91]:
# зависимость между полом заёмщика и случаями задолженности
gender_f = {
        'M' : 'Мужчина',
        'F' : 'Женщина',
        'XNA' : 'Неопределено'
}

In [92]:
data_gender = dep('gender').data
data_gender.insert(0, 'gender_name', [gender_f.get(x) for x in data_gender.index])
data_gender = data_gender.style.format({'ratio':'{:.2%}', 'div_from_total':'{:.2%}'})
data_gender

Total debt ratio 8.11%


,gender_name,debt,total_cases,ratio,div_from_total
gender,,,,,
F,Женщина,994,14181,7.01%,-1.10%
M,Мужчина,747,7279,10.26%,2.15%
XNA,Неопределено,0,1,0.00%,-8.11%


In [93]:
data_incom_tp = dep('income_type')
data_incom_tp

Total debt ratio 8.11%


,debt,total_cases,ratio,div_from_total
income_type,,,,
безработный,1,2,50.00%,41.89%
в декрете,1,1,100.00%,91.89%
госслужащий,86,1457,5.90%,-2.21%
компаньон,376,5078,7.40%,-0.71%
пенсионер,216,3834,5.63%,-2.48%
предприниматель,0,2,0.00%,-8.11%
сотрудник,1061,11086,9.57%,1.46%
студент,0,1,0.00%,-8.11%


**Вывод**

Проведен анализ случаем задолженности по всем представленным в данных категориям.

## Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [94]:
data_children

,debt,total_cases,ratio,div_from_total
children,,,,
0,1063,14099,7.54%,-0.57%
1,445,4855,9.17%,1.05%
2,202,2128,9.49%,1.38%
3,27,329,8.21%,0.09%
4,4,41,9.76%,1.64%
5,0,9,0.00%,-8.11%


**Вывод**

По представленным данным можно сделать вывод об присутствии/отсутствии зависимости между случаями задолженности и одной стороны и отсутствием/наличием 1-2-3 детей с другой стороны. Данные по клиентам с 4 или 5 детьми представлены в небольшом, относительно всего объема данных, количестве и исследования по ним не могут быть интерпретированы.

На основании проведенного исследования можно утверждать, что клиенты без детей допускают просрочки в меньшей степени, чем клиенты с детьми. Самыми недисциплинированными, на основании представленных данных, оказались клиенты с 2 детьми. Так, доля случаем задолжености в такой группе на 1.95 п.п. выше, чем по группе без детей.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [95]:
data_family_st

,debt,total_cases,ratio,div_from_total
family_status,,,,
в разводе,85,1194,7.12%,-0.99%
вдовец / вдова,63,958,6.58%,-1.54%
гражданский брак,388,4160,9.33%,1.21%
женат / замужем,931,12340,7.54%,-0.57%
не женат / не замужем,274,2809,9.75%,1.64%


**Вывод**

На основании проведенного исследования можно утверждать, что наименее финансово дисциплинированными являются клиенты категории "не женат / не замужем", и люди состоящие в гражданском браке. Разница между долями по наименее(не женат / не замужем) и наиболее(вдовец / вдова) дисциплинированными  группам составляет 3.17 п.п.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [96]:
data_incom

,incom_level,debt,total_cases,ratio,div_from_total
incom_id,,,,,
1,до 100000,354,4463,7.93%,-0.18%
2,от 100001 до 130000,328,3871,8.47%,0.36%
3,от 130001 до 160000,391,4407,8.87%,0.76%
4,от 160001 до 220000,384,4712,8.15%,0.04%
5,свыше 220001,284,4008,7.09%,-1.03%


**Вывод**

На основании проведенного исследования можно утверждать, что наиболее финансово дисциплинированными являются клиенты с уровнем дохода более 220001. Интересно, что худший результат показала группа не с минимальным доходом, а средняя группа клиентов с доход от 130001 до 160000. Разброс по группам 1.78 п.п.

### Как разные цели кредита влияют на его возврат в срок?

In [97]:
data_purpose

,debt,total_cases,ratio,div_from_total
purpose,,,,
автомобиль,403,4308,9.35%,1.24%
недвижимость,648,8889,7.29%,-0.82%
недвижимость ком.,99,1311,7.55%,-0.56%
образование,370,4014,9.22%,1.11%
ремонт,35,606,5.78%,-2.34%
свадьба,186,2333,7.97%,-0.14%


**Вывод**

На основании проведенного исследования можно утверждать, что наиболее финансово дисциплинированными являются клиенты берущие кредит на ремонт. Выделенная отдельно группа по операциям с комменческой недвижимостью, имеет показатели близкие к группе 'недвижимость', разница всего 0.26 п.п. Больше всего проблем испытывают клиенты, взявшие кредит на приобретение автомобиля.  Разброс по группам 3.57 п.п.

## Выводы

Исследование завершено, ответы на поставленные вопросы получены. Из заявленных к анализу зависимостей, наиболее значимая, т.е. имеющая больший разброс значений ро группам - цель кредита. Наименее значимая, как не странно - уровень дохода.

Анализ за рамками поставленной задачи показал, что на случаи задолженности оказываем существенное влияние пол клиента, женщины на 3.25 п.п. более аккуратны. Если рассматривать заёмщиков с точки зрения занятости, то из представленных категорий (за исключением 'в декрете', 'безработный', 'студент', 'предприниматель' в виду с низкой представленнностью в данных) наиболее аккуратны пенсионеры, а менее аккуратны сотрудники. Разница между указанными категориями 3.94 п.п., что является самым большим разбросом в данном исследовании.